<a href="https://colab.research.google.com/github/kushv16/Audio-Sentiment-Analysis/blob/master/Testing_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
     

Mounted at /content/drive/


In [ ]:
!pip install noisereduce

import pandas as pd
import numpy as np
import librosa
import librosa.display
import random
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import noisereduce as nr
%matplotlib inline
%config InlineBackend.figure_format='retina'
import warnings
warnings.filterwarnings("ignore")
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # to avoid TensorFlow compiler flag message

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import (Conv2D, BatchNormalization, Dropout, Flatten, Dense, MaxPool2D)
from tensorflow.keras import initializers

# Create convolutional neural network and return summary
model = keras.Sequential()
model.add(Conv2D(filters=256, kernel_size=5, strides=(2, 2), activation="relu", input_shape=(30,200,1)))
model.add(MaxPool2D(pool_size=2))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=4, strides=(2, 1), activation="relu"))
model.add(MaxPool2D(pool_size=2))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=7, activation="softmax"))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 13, 98, 256)       6656      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 49, 256)       0         
 2D)                                                             
                                                                 
 batch_normalization_7 (Batc  (None, 6, 49, 256)       1024      
 hNormalization)                                                 
                                                                 
 conv2d_7 (Conv2D)           (None, 2, 46, 128)        524416    
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 1, 23, 128)       0         
 2D)                                                             
                                                      

In [ ]:
from scipy.io import wavfile
path = ''

def clean_files(audio_file_path): 
  y, sr = librosa.load(audio_file_path, sr=16000)
  y_trim, _ = librosa.effects.trim(y, top_db=20)
  y_noise_rem = nr.reduce_noise(y=y_trim, sr=sr, prop_decrease=0.1, stationary=True)
  return y_noise_rem, sr

In [ ]:
y, sr = clean_files("/content/drive/MyDrive/LY Project/Testing Data/Clips/Beach/Actor_Divesh/beach_sad.wav")

In [ ]:
# y, sr = librosa.load("/content/drive/MyDrive/LY Project/police_happy_8000_cleaned.wav", sr=16000)
mfccs = []

mfccs.append(librosa.feature.mfcc(y=y, sr=sr, fmin=50, n_mfcc=30))

  # Define function to resize the 2D arrays
def resize_array(array):
    new_matrix = np.zeros((30,200))   # Initialize the new matrix shape with an array 30X200 of zeros
    for i in range(30):               # Iterate rows
        for j in range(200):          # Iterate columns
            try:                                 # the mfccs of a sample will replace the matrix of zeros, then cutting the array up to 200
                new_matrix[i][j] = array[i][j]
            except IndexError:                   # if mfccs of a sample is shorter than 200, then keep looping to extend lenght to 200 with 0s
                pass
    return new_matrix

# Create a variable to store the new resized mfccs and apply function for all the extracted mfccs
resized_mfccs = []

for mfcc in mfccs:
    resized_mfccs.append(resize_array(mfcc))

In [ ]:
model.load_weights('/content/drive/MyDrive/LY Project/Result and Analysis/CNN Results | 2D MFCCs | Without CremaD/2 Layers/cnn_two_layers_weights.h5')

In [ ]:
x = np.array(resized_mfccs).reshape((30,200))

x_tr = np.load('/content/drive/MyDrive/LY Project/Result and Analysis/CNN Results | 2D MFCCs | Without CremaD/x_tr.npy')

tr_mean = np.mean(x_tr, axis=0)
tr_std = np.std(x_tr, axis=0)

x = (x - tr_mean)/tr_std
x = np.array(x).reshape((1,30,200,1))

In [ ]:
labels = {'angry':0,'disgust':1,'fear':2,'happy':3,'neutral':4,'sad':5,'surprise':6}

y_pred = model.predict(x)

print(np.argmax(y_pred))

1/1 [==============================] - 0s 31ms/step
4


In [ ]:
y_pred

array([[0.03333402, 0.11720824, 0.03435962, 0.0612562 , 0.5043508 ,
        0.24735655, 0.00213456]], dtype=float32)